In [ ]:
# ATTENTION: execute more than once create non-needed directory owing to current directory change
!git clone https://github.com/bird0401/Instance_level_recognition.git

In [ ]:
%cd /content/Instance_level_recognition/app/ml
%ls

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
! echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" |sudo tee /etc/apt/sources.list.d/gcsfuse.list
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! apt-get -y -q update
! apt-get -y -q install gcsfuse

In [ ]:
! mkdir -p ./data
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 entity_dogs ./data

In [ ]:
# get file paths
import glob
l = glob.glob('/content/Instance_level_recognition/app/ml/data/imgs/**/*')

In [ ]:
df = pd.DataFrame(l, columns=["path"])
df.head()

In [ ]:
def get_wikidata_id(path):
  return path.split("/")[-2]

In [ ]:
df['wikidata_id'] = df['path'].apply(get_wikidata_id)
df.head()

In [ ]:
# extract only label which has more than or equal to 3 images 
vc = df['wikidata_id'].value_counts()
used_wikidata_id = vc[vc >= 3].index
df = df[df["wikidata_id"].isin(used_wikidata_id)]

In [ ]:
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['wikidata_id'])
df.head()

In [ ]:
with open("le.pkl", "wb") as fp:
    joblib.dump(encoder, fp)
len(df.label)

In [ ]:
df.to_csv("/content/Instance_level_recognition/app/ml/data/path_label.csv")

In [ ]:
train_val_indices, test_indices = train_test_split(list(range(len(df.label))), test_size=0.2, stratify=df.label)
df.loc[train_val_indices, "is_train_val"] = 1
df.loc[test_indices, "is_train_val"] = 0
df_train, df_test = df[df["is_train_val"] == 1].reset_index(), df[df["is_train_val"] == 0].reset_index()
len(df_train), len(df_test)

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'])
for fold, ( _, val_) in enumerate(skf.split(X=df_train, y=df_train.label)):
      df_train.loc[val_ , "kfold"] = fold
df_train.head()

In [ ]:
df_train.to_csv("/content/Instance_level_recognition/app/ml/data/train.csv")
df_test.to_csv("/content/Instance_level_recognition/app/ml/data/test.csv")